# FAISS DB
- Facebook AI Similarity Search

In [16]:
# !pip install faiss-cpu

### 영화 추천 시스템

In [17]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

In [18]:
movies = [
    {"id": 0, "title": "인셉션", "discription": "꿈속에서 다른 사람의 생각을 조작하는 스토리"},
    {"id": 1, "title": "인터스텔라", "discription": "인류의 생존을 위해 우주를 탐험하는 이야기"},
    {"id": 2, "title": "어벤져스: 엔드게임", "discription": "사라진 인류를 되찾기 위한 히어로들의 마지막 전투"},
    {"id": 3, "title": "기생충", "discription": "한 가족이 부잣집에 위장 취업하며 벌어지는 블랙코미디"},
    {"id": 4, "title": "타이타닉", "discription": "운명적인 사랑과 비극적인 배 침몰 사고를 그린 이야기"},
    {"id": 5, "title": "쇼생크 탈출", "discription": "억울하게 수감된 남자의 감동적인 탈옥과 희망의 이야기"},
    {"id": 6, "title": "아바타", "discription": "외계 행성에서 벌어지는 인간과 나비족의 갈등과 공존"},
    {"id": 7, "title": "조커", "discription": "한 남자의 고통과 분노가 세상을 뒤흔드는 광기로 이어지는 이야기"},
    {"id": 8, "title": "라라랜드", "discription": "꿈을 좇는 두 남녀의 아름답고도 슬픈 로맨스 뮤지컬"},
    {"id": 9, "title": "매드맥스: 분노의 도로", "discription": "디스토피아 세계에서 자유를 찾아 질주하는 생존기"},
    {"id": 10, "title": "겨울왕국", "discription": "얼음을 다루는 여왕과 자매의 사랑과 모험"},
    {"id": 11, "title": "너의 이름은", "discription": "운명처럼 서로의 몸이 바뀐 두 청춘의 시간 여행"},
    {"id": 12, "title": "부산행", "discription": "좀비 바이러스가 퍼진 한국에서 살아남기 위한 열차 안의 사투"},
    {"id": 13, "title": "반지의 제왕", "discription": "절대 반지를 파괴하기 위한 전설적인 모험과 전쟁"},
    {"id": 14, "title": "해리 포터와 마법사의 돌", "discription": "마법 학교에 입학한 소년 해리의 첫 마법사 모험"},
    {"id": 15, "title": "캐치 미 이프 유 캔", "discription": "천재적인 사기꾼과 그를 쫓는 FBI 요원의 심리전"},
    {"id": 16, "title": "셜록 홈즈", "discription": "천재 탐정 셜록과 왓슨의 두뇌 싸움과 사건 해결"},
    {"id": 17, "title": "킹스맨: 시크릿 에이전트", "discription": "비밀 요원 조직에서 벌어지는 세련되고 유쾌한 첩보 액션"},
    {"id": 18, "title": "1917", "discription": "1차 세계대전 속 두 병사의 긴박한 임무 수행기"},
    {"id": 19, "title": "미드소마", "discription": "낮이 끝나지 않는 축제에서 벌어지는 끔찍한 의식과 공포"}
]


In [19]:
descriptions = [movie['discription'] for movie in movies]
desc_embed = np.array([model.encode(description) for description in descriptions], dtype=np.float32)

In [20]:
dim = desc_embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(desc_embed)

In [21]:
query_text = '조커'
query_embed = np.array([model.encode(query_text)], dtype='float32')

In [22]:
top_n = 5
distances, indices = idx.search(query_embed, top_n)

In [23]:
print(distances)
print(indices)

[[0.6698791 1.0112237 1.044498  1.1335833 1.1763483]]
[[ 3  8  0  2 14]]


In [24]:
for i in range(top_n):
    movie_id = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_id]['title']} (유사도 거리: {distances[0][i]:.2f})")

1번째 추천: 기생충 (유사도 거리: 0.67)
2번째 추천: 라라랜드 (유사도 거리: 1.01)
3번째 추천: 인셉션 (유사도 거리: 1.04)
4번째 추천: 어벤져스: 엔드게임 (유사도 거리: 1.13)
5번째 추천: 해리 포터와 마법사의 돌 (유사도 거리: 1.18)


### 사용자 맞춤 뉴스 추천 <실습>

In [38]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 뉴스의 제목을 20 크롤링 (뷰티풀숲 사용)
titles = []
query = "llm"
for start in [1, 11]:
    url = f"https://search.naver.com/search.naver?where=news&query={query}&start={start}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.select("a.bynlPWBHumGsbotLYK9A.jT1DuARpwIlNAFMacxlu")
    for a in a_tags:
        title = a.get_text(strip=True)
        if title not in titles:
            titles.append(title)
        if len(titles) >= 20:
            break
    if len(titles) >= 20:
        break

print(len(titles))
for i in range(len(titles)):
    print(f"- {titles[i]}")

user_query = input("키워드를 입력하세요: ")
    
# FAISS 활용해 맞춤 뉴스 추천
embed = np.array(model.encode(titles), dtype=np.float32)
dim = embed.shape[1]
idx = faiss.IndexFlatL2(dim)
idx.add(embed)
query_embed = np.array([model.encode(user_query)], dtype='float32')
top_n = 5
distances, indices = idx.search(query_embed, top_n)
for i in range(top_n):
    id = indices[0][i]
    print(f"{i+1}번째 추천: {titles[id]} (유사도 거리: {distances[0][i]:.2f})")

20
- AWS "韓 데이터 수집·정제 미흡…토종LLM협업 확대할 것"
- 1.8조 AI 추경…GPU 확보·월드 베스트LLM닻 올린다
- [오늘의 DT인] "뛰어난LLM이라도 실질적인 업무 처리로 이어지지 않으...
- '월드 베스트LLM' 프로젝트 본격 가동
- KT, 국내 기업 최초 'LLM플랫폼' 수출…태국 특화 모델 개발
- '해외에 태극기 꽂는다'…엔비디아·MS가 반한 韓 AI 기업은? [김성태의...
- 한국형 챗GPT 1개사에 몰아줘야…승부는 '산업 AI'로
- KT, 국내 최초LLM플랫폼 수출 완료
- KT, 태국에LLM플랫폼 수출
- KT, 첫 해외 진출…태국에 AX 플랫폼 수출한다
- "올해 AI가 인간능력 넘어설 것"…업스테이지, '솔라' 글로벌 출사표
- 투게더AI, 플랫폼 업글...오픈소스LLM파인튜닝 진입 장벽 낮춘다
- 엑사포스, 7500만달러 투자 유치...LLM·행동 모델 결합해 보안 운영 자...
- 파운데이션 EGI, 760만달러 투자 유치..."LLM으로 제조 공정 자동화 혁신...
- KT, '지니TV'에 GPT-4o 기반 '한국형 AI에이전트' 심는다
- 업스테이지, '경량LLM' 글로벌 1위 도전…'솔라 1.5' 6월 공개
- KT completes project to create ThaiLLMwith Thailand
- LGU+, 국회 과방위와 AI 기술력 강화 논의…엑사원·익시젠 등 시연
- "한국어LLM신뢰성 강화"…NIA-TTA, 학습용 데이터 유해 표현 검출 모델...
- 'LLM늦었지만 제조는 기회의 땅'…산업AI 급부상, 왜
1번째 추천: 파운데이션 EGI, 760만달러 투자 유치..."LLM으로 제조 공정 자동화 혁신... (유사도 거리: 1.12)
2번째 추천: 엑사포스, 7500만달러 투자 유치...LLM·행동 모델 결합해 보안 운영 자... (유사도 거리: 1.29)
3번째 추천: '월드 베스트LLM' 프로젝트 본격 가동 (유사도 거리: 1.45)
4번째 추천: KT, 태국에LLM플랫폼 수출 (유사도